In [ ]:
%%capture
!python -m pip install --upgrade pip
!pip install --upgrade scikit-learn-intelex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
import sys
import os
import site
import pandas as pd
import numpy as np

sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))

In [ ]:
RANDOM_STATE = 42

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/telecom_churn.csv")
df.head()

In [ ]:
np.random.seed(RANDOM_STATE)

Сделаем первичную обработку данных:

In [ ]:
df = df.drop(columns = 'State')
df['Voice mail plan']= df['Voice mail plan'].map({'Yes': 1, 'No': 0})
df['International plan']= df['International plan'].map({'Yes': 1, 'No': 0})
df['Churn'] = df['Churn'].astype(int)


In [ ]:
print(df.head())

In [ ]:
X = df.drop(columns = 'Churn')
y = df['Churn']

По кросс-валидации с тремя фолдами и метрикой ROC-AUC обучим логистическую регрессию, метод опорных векторов и решающее дерево

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

cross_val_score(SVC(kernel='linear'), X, y, cv=3, scoring='roc_auc', verbose=2).mean()

In [ ]:
cross_val_score(LogisticRegression(), X, y, cv=3, scoring='roc_auc', verbose=2).mean()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
cross_val_score(DecisionTreeClassifier(random_state = RANDOM_STATE), X, y, cv=3, scoring='roc_auc',  verbose = 2).mean()

Улучшим модели подобрав гиперпараметры на трейне

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C' : np.linspace(0.1, 10.01, 20)}

gs = GridSearchCV(LogisticRegression(), params, cv=3, scoring='roc_auc')

gs.fit(Xtrain, ytrain)

gs.best_score_, gs.best_estimator_

In [ ]:
from sklearn.metrics import roc_auc_score

lr = gs.best_estimator_

pred_lr = lr.predict_proba(Xtest)[:,1]
print(roc_auc_score(ytest, pred_lr))

In [ ]:

params = {'criterion': ['gini','entropy', 'log_loss'], 'max_depth' : np.arange(0, 50, 1),'min_samples_split':np.arange(2, 10, 2), 'min_samples_leaf': np.arange(0, 50, 1)}

gs = GridSearchCV(DecisionTreeClassifier(random_state = RANDOM_STATE), params, cv=3, scoring='roc_auc')

gs.fit(Xtrain, ytrain)

gs.best_score_, gs.best_estimator_


lr = gs.best_estimator_

pred_lr = lr.predict_proba(Xtest)[:,1]
print(roc_auc_score(ytest, pred_lr))

Нарисуем roc-кривую для наилучшей модели и посчитаем f1_score

In [ ]:
from sklearn.metrics import roc_curve, auc
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score

pred = lr.predict(Xtest)
print(f1_score(ytest, pred))
print(gs.best_estimator_)

fpr, tpr, threshold = roc_curve(ytest, pred_lr)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, 'b')
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.show()

Сравните между собой две лучшие модели.

In [ ]:
!pip install -q dtreeviz
import dtreeviz

In [ ]:
model = dtreeviz.model(lr, Xtrain, ytrain, feature_names=Xtrain.columns)
model.view(fancy = False, scale = 2)